# interpretation of drowsy detection

process方法实现了疲劳检测的核心算法
1. 处理输入帧：通过mp处理当前的视频帧，提取面部的关键点
2. 计算EAR：计算眼睛的EAR指标以检测疲劳状态
3. 管理疲劳时间：累计疲劳时间，并在时间超过设定值的时候播放警报
4. 动态更新状态：根据EAR指标更新状态和界面信息
5. 可视化信息：在视频帧上绘制与检测相关的文本和眼睛关键点

# streamlit_app

In [ ]:
with col1:
    ctx = webrtc_streamer(
        key="drowsiness-detection",
        # key = "local-stream",
        video_frame_callback=video_frame_callback,
        audio_frame_callback=audio_frame_callback,
        # rtc_configuration={"iceServers": [{"urls": ["stun:stun.l.google.com:19302"]}]},  # Add this to config for cloud deployment.
        media_stream_constraints={"video": {"height": {"ideal": 480}}, "audio": True},
        # media_stream_constraints={"video": True, "audio": True},
        video_html_attrs=VideoHTMLAttributes(autoPlay=True, controls=False, muted=False),
    )

这段代码主要是实现了一个视频流实时处理的框架，流程如下：
1. 视频流的启动：
   - webrtc_streamer是Streamlit-webrtc提供的一个组件，用于通过WebRTC处理实时视频和音频流
   - 他启动了一个WebRTC会话，捕获视频流并将每一帧传递到指定的回调函数中
2. 视频流处理：
   - video_frame_callback是视频流处理的回调函数，webtrc_streamer捕获到的视频帧会传递给他进行处理。
   - 在video_frame_callback中，我们会调用video_handler的process去处理帧
     - process方法对帧进行计算
     - 返回处理后的帧以及是否需要播放警报的标志

- lock: 使用锁来保护共享状态的更新，确保线程安全

## 三个模块之间的Link

1. webtrc_streamer:
   - 核心作用：捕获音视频流并处理他们
   - 调用video_frame_callback来处理每一帧的视频
   - 最终的处理结果会通过webrtc实时传输回客户端
2. video_frame_callback:
   - 接受webtrc_streamer传来的每一帧视频，进行处理
   - 利用video_handler来完成具体的业务逻辑
3. video_handler
   - 专门负责对于单帧视频的核心处理（eye/mouth）
   - process方法封装了业务逻辑，如各种计算以及返回处理结果
- webtrc_streamer是连接用户设备摄像头/麦克风和程序逻辑之间的brige
- 他不断地从用户设备获取视频帧，将帧传递给video_frame_callback处理，然后video_frame_callback将处理后的帧返回到用户设备显示